# Solving a Sudoku with AI
This notebook was made for solving the quizes and exercies in the AI nanodegree from Udacity.

## Coding the board

In [72]:
rows = 'ABCDEFGHI'
cols = '123456789'
possible_digits = '123456789'

def cross(a, b):
    """
    Return a list with all concatenations of a letter in
    `a` with a letter in `b`
    
    Args:
        a: A string
        b: A string
    Returns:
        A list formed by all the possible concatenations of a
        letter in `a` with a letter in `b`
    """
    return [s + t for s in a for t in b]

boxes = cross(rows, cols)
print(boxes)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']


In [100]:
# Lets get all the row units
# row_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
row_units = [cross(row, cols) for row in rows]

# Same for column units
# column_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
column_units = [cross(rows, col) for col in cols]

# And now for square units
# square_units[0] = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']
square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]

diagonals = [['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9'],
            ['A9', 'B8', 'C7', 'D6', 'E5', 'F4', 'G3', 'H2', 'I1']]

unitlist = row_units + column_units + square_units + diagonals
units = dict((box, [unit for unit in unitlist if box in unit]) for box in boxes)
peers = dict((box, set(sum(units[box], []))-set([box])) for box in boxes)

print('First row unit: ', row_units[0])
print('First col unit: ', column_units[0])
print('First sqr unit: ', square_units[0])

First row unit:  ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
First col unit:  ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
First sqr unit:  ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']


In [101]:
def display(values):
    """
    Prints the values of a sudoku as a 2-D grid.
    
    Args:
        values: A dict representing a sodoku.
        
    Returns:
        `None`
    """
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join(['-' * (width * 3)] * 3)
    for row in rows:
        print(''.join(values[row + col].center(width) + ('|' if col in '36' else '') for col in cols))
        if row in 'CF': print(line)
    return

def grid_values(grid):
    """
    Returns a dict that represents a sudoku.
    
    Args:
        grid: A string with the starting numbers
        for all the boxes in a sudoku. Empty boxes
        can be represented as dots `.`.
        Example: `'..3.2.6.'...`
        
    Returns:
        A dict that represents a sudoku. The keys
        will be the boxes labels and it's value will be the number
        or a dot `.` if the box is empty.
    """
    assert len(grid) == 81, "The lenght of `grid` should be 81. A 9x9 sudoku"
    chars = []
    for char in grid:
        if char in possible_digits: chars.append(char)
        if char == '.': chars.append(possible_digits)
    return dict(zip(boxes, chars))



sudoku_starting_dict = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
display(sudoku_starting_dict)


123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

## Strategy 1: Elimination
If a box has a value assigned, the none of the peers of this box can have this value
![Image from udacity AI nanodegree](https://i.imgur.com/GID0eWu.png)

In [102]:
def eliminate(sudoku_dict):
    """
    Returns a sudoku dict after applying the eliminate technique.
    
    Args:
        sudoku_dict: A dict representing the sudoku. It'll contain
        in each box the value of it, or the possible values.
        
    Returns:
        A sudoku dict after applying the eliminate technique in all
        the boxes.
    """
    solved_values = [box for box in sudoku_dict.keys() if len(sudoku_dict[box]) == 1]
    for box in solved_values:
        value = sudoku_dict[box]
        for peer in peers[box]:
            sudoku_dict[peer] = sudoku_dict[peer].replace(value, '')
            
    return sudoku_dict

sudoku_after_eliminate = eliminate(sudoku_starting_dict)
print('\n Sudoku after eliminate technique. \n')
display(sudoku_after_eliminate)


 Sudoku after eliminate technique. 

   4     4578    3   |  149     2     147  |   6     5789    5   
   9     2467    47  |   3      47     5   |   78     8      1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
 12345  12345    8   |         3456        |   9    134567 34567 
   7    123459   49  |  1459   369    124  |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6     478     9   |   5     1478    47  
   8      16     47  |   2     457     3   |   17    467     9   
   6     4679    5   |   4      1      47  |   3    24678   2467 


## Strategy 2: Only Choice
If there is a box in a unit which would allow a certain digit, then that bux must be assigned that digit
![Image from udacity AI nanodegree](https://i.imgur.com/G3ACj8v.png)

In [103]:
def only_choice(sudoku_dict):
    """
    It runs through all the units of a sudoku
    and it applies the only choice technique.
    
    Args: 
        sudoku_dict: A dict representing the sudoku.
    Returns:
        A sudoku dict after applying the only choice technique.
    """
    for unit in unitlist:
        for value in possible_digits:
            value_places = [box for box in unit if value in sudoku_dict[box]]
            if len(value_places) == 1:
                sudoku_dict[value_places[0]] = value
                
    return sudoku_dict

sudoku_after_only_choice= only_choice(sudoku_after_eliminate)
display(sudoku_after_only_choice)

   4      8      3   |   9      2      1   |   6      9      5   
   9      2      47  |   3      4      5   |   8      8      1   
   25    257     1   |   8      7      6   |   4      2      3   
---------------------+---------------------+---------------------
 12345  12345    8   |         3456        |   9    134567 34567 
   7    123459   9   |   5      3      2   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6      8      9   |   5     1478    47  
   8      6      47  |   2      5      3   |   17     6      9   
   6      9      5   |   4      1      7   |   3      8      2   


## Stategy: Naked twins
![](https://i.imgur.com/HPMZ5OC.png)

In [105]:
def get_pairs(sudoku):
    """Return all the boxes with just 2 digits"""
    return [box for box in boxes if len(sudoku[box]) == 2]

def naked_twins(sudoku):
    """Eliminates values using the naked twins strategy.
    
    Args:
        sudoku: A dict representation of the sudoku
        
    Returns:
        A dict representation of the sudoku with the
        naked twins deleted from it's peers.
    """
    pair_list = get_pairs(sudoku)
    for box in pair_list:
        for unit in units[box]:
            # Find the peers in the unit that contains 2 digits
            # but is not the box itself
            peers_with_pairs = set(unit).intersection(set(peers[box])).intersection(set(pair_list))
            
            for peer in peers_with_pairs:
                if sudoku[box] == sudoku[peer]:
                    for item in set(unit).difference(set([box, peer])):
                        digit_1 = sudoku[box][0]
                        digit_2 = sudoku[box][1]
                        
                        if digit_1 in sudoku[item]:
                            sudoku[item] = sudoku[item].replace(digit_1, '')
                        if digit_2 in sudoku[item]:
                            sudoku[item] = sudoku[item].replace(digit_2, '')
    return sudoku

before_naked_twins = {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8', 'H5': '6', 'F9': '7', 'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8', 'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23', 'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5', 'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'A4': '2357', 'A7': '27', 'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6', 'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2', 'F6': '125', 'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '379', 'F1': '6', 'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'E2': '37', 'F7': '35', 'F8': '9', 'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17', 'D3': '2379', 'B4': '27', 'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'D6': '279', 'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}

after_naked_twins = {'G7': '6', 'G6': '3', 'G5': '2', 'G4': '9', 'G3': '1', 'G2': '8', 'G1': '7', 'G9': '5', 'G8': '4', 'C9': '1', 'C8': '5', 'C3': '8', 'C2': '237', 'C1': '23', 'C7': '9', 'C6': '6', 'C5': '37', 'A4': '2357', 'A9': '8', 'A8': '6', 'F1': '6', 'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'F6': '125', 'F7': '35', 'F8': '9', 'F9': '7', 'B4': '27', 'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'C4': '4', 'B8': '3', 'B9': '4', 'I9': '9', 'I8': '7', 'I1': '23', 'I3': '23', 'I2': '6', 'I5': '5', 'I4': '8', 'I7': '1', 'I6': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'E8': '1', 'A7': '27', 'A6': '257', 'E5': '347', 'E4': '6', 'E7': '345', 'E6': '579', 'E1': '8', 'E3': '79', 'E2': '37', 'H8': '2', 'H9': '3', 'H2': '9', 'H3': '5', 'H1': '4', 'H6': '17', 'H7': '8', 'H4': '17', 'H5': '6', 'D8': '8', 'D9': '6', 'D6': '279', 'D7': '34', 'D4': '237', 'D5': '347', 'D2': '1', 'D3': '79', 'D1': '5'}

display(naked_twins(before_naked_twins))
print('\n')
display(after_naked_twins)
print('/n', 'Are they equal: ', len(set(naked_twins(before_naked_twins)).difference(set(after_naked_twins))))

  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1    79 | 237  347  279 |  34   8    6  
  8    37   79 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  


  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1    79 | 237  347  279 |  34   8    6  
  8    37   79 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8   

## Constraint Propagation
Is a technique that uses local constrains in a space in order to reduce the search space. It'll reduce the number of possibilities.  

In [106]:
def solved_boxes(sudoku):
    """Returns the number of boxes solved in a sudoku"""
    return len([box for box in sudoku.keys() if len(sudoku[box]) == 1])

def reduce_puzzle(sudoku):
    """
    Uses constrain propagation to reduce the search space
    
    Args:
        sudoku_dict: A dict representing the sudoku.
    
    Returns:
        A sudoku dict solved or partially solved
    """
    improving = True
    while improving:
        solved_values = solved_boxes(sudoku)
        
        sudoku = eliminate(sudoku)
        sudoku = only_choice(sudoku)
        sudoku = naked_twins(sudoku)
        
        solved_values_after = solved_boxes(sudoku)
        
        improving = solved_values != solved_values_after
        
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in sudoku.keys() if len(sudoku[box]) == 0]):
            return False
    return sudoku

sudoku = reduce_puzzle(sudoku_starting_dict)
display(sudoku)

TypeError: 'bool' object is not subscriptable

## Stategy 3: Search
Now lets use Depth first search to find a solution for harder sudokus

In [107]:
grid_2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
sudoku_2 = grid_values(grid_2)

def update_dict(d, key, value):
    """Updates the dict `d` and returns the dict"""
    d.update({key: value})
    return d

def some(seq):
    """Return some element of `seq` that is `True` http://norvig.com/sudoku.html"""
    for e in seq:
        if e: return e
    return False

def search(sudoku):
    """
    Uses depth search first and propagaation to find a solution for the sudoku
    
    
    Args:
        sudoku: A dict representation of a sudoku
    
    Returns:
        A dict representation of the sudoku solved
    """
    sudoku = reduce_puzzle(sudoku)
    if sudoku is False:
        return False
    
    # Check if the sudoku is solved
    if all([len(sudoku[box]) == 1 for box in boxes]):
        return sudoku
    
    n, min_box = min((len(sudoku[box]), box) for box in boxes if len(sudoku[box]) > 1)
    
    return some(search(update_dict(sudoku.copy(), min_box, digit)) for digit in sudoku[min_box])
    
display(search(sudoku_2))

TypeError: 'bool' object is not subscriptable

In [111]:
diagonal_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
solved_diag_sudoku = {'G7': '8', 'G6': '9', 'G5': '7', 'G4': '3', 'G3': '2', 'G2': '4', 'G1': '6', 'G9': '5', 'G8': '1', 'C9': '6', 'C8': '7', 'C3': '1', 'C2': '9', 'C1': '4', 'C7': '5', 'C6': '3', 'C5': '2', 'C4': '8', 'E5': '9', 'E4': '1', 'F1': '1', 'F2': '2', 'F3': '9', 'F4': '6', 'F5': '5', 'F6': '7', 'F7': '4', 'F8': '3', 'F9': '8', 'B4': '7', 'B5': '1', 'B6': '6', 'B7': '2', 'B1': '8', 'B2': '5', 'B3': '3', 'B8': '4', 'B9': '9', 'I9': '3', 'I8': '2', 'I1': '7', 'I3': '8', 'I2': '1', 'I5': '6', 'I4': '5', 'I7': '9', 'I6': '4', 'A1': '2', 'A3': '7', 'A2': '6', 'E9': '7', 'A4': '9', 'A7': '3', 'A6': '5', 'A9': '1', 'A8': '8', 'E7': '6', 'E6': '2', 'E1': '3', 'E3': '4', 'E2': '8', 'E8': '5', 'A5': '4', 'H8': '6', 'H9': '4', 'H2': '3', 'H3': '5', 'H1': '9', 'H6': '1', 'H7': '7', 'H4': '2', 'H5': '8', 'D8': '9', 'D9': '2', 'D6': '8', 'D7': '1', 'D4': '4', 'D5': '3', 'D2': '7', 'D3': '6', 'D1': '5'}


display(search(grid_values(diagonal_grid)))
print('\n')
display(solved_diag_sudoku)
print('\n', 'Are they equal: ', not len(set(search(grid_values(diagonal_grid))).difference(set(solved_diag_sudoku))))

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 

 Are they equal:  True
